In [1]:
import pandas as pd
import collections
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('ds_sentiment/annotation_set_sentiment.csv')

In [3]:
df_pp = df

### tokenization

In [4]:
# Convert all text to lowercase.
df_pp['body-sm'] = df_pp['body'].str.lower()

In [5]:
#remove some more characters. This should be done in the general preprocessing before!!
df_pp['body-sm'] = df_pp['body-sm'].str.replace('\n', '')
df_pp['body-sm'] = df_pp['body-sm'].str.replace('&#x200b;', '')

In [6]:
#removing nulls?
mask = df_pp['body-sm'].notnull()
df_pp = df_pp[mask]
print(len(df_pp))

468


In [7]:
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

df_pp['body-sm'] = df_pp['body-sm'].apply(tokenize_text)

### Stopwords

In [8]:
# Remove stopwords using the NLTK library.
stopwords = stopwords.words('english')
df_pp['body-sm'] = df_pp['body-sm'].apply(lambda x: [word for word in x if word not in stopwords])

### lemmatizer

In [9]:
lemmatizer = WordNetLemmatizer()
df_pp['body-sm'] = df_pp['body-sm'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])

In [10]:
df_pp['body-sm'] = df_pp['body-sm'].apply(lambda x: ' '.join(x))

In [11]:
# # Remove punctuation using the string library.
df_pp['body-sm'] = df_pp['body-sm'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [12]:
#testing
word = "cars"
lemma_word = lemmatizer.lemmatize(word)

print(f"Original word: {word}")
print(f"Lemmatized word: {lemma_word}")


Original word: cars
Lemmatized word: car


### add another layer of textblob sentiment analysis

In [13]:
from textblob import TextBlob

In [14]:
sentiment_tb_df = df_pp

In [15]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
  
#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity


#Create two new columns ‘Subjectivity’ & ‘Polarity’
sentiment_tb_df['sentiment_textblob_subjectivity-v2'] = sentiment_tb_df['body-sm'].apply(getSubjectivity)
sentiment_tb_df['sentiment_textblob_polarity-v2'] = sentiment_tb_df['body-sm'].apply(getPolarity)

sentiment_tb_df['sentiment_textblob-v2'] = 'neutral'
sentiment_tb_df.loc[df_pp['sentiment_textblob_polarity-v2'] > 0, 'sentiment_textblob-v2'] = 'positive'
sentiment_tb_df.loc[df_pp['sentiment_textblob_polarity-v2'] < 0, 'sentiment_textblob-v2'] = 'negative'
 

In [16]:
sentiment_tb_df = sentiment_tb_df.drop('sentiment_textblob_subjectivity-v2', axis=1)
sentiment_tb_df = sentiment_tb_df.drop('sentiment_textblob_polarity-v2', axis=1)

In [18]:
sentiment_tb_df.to_csv('ds_sentiment/annotation_set_sentiment_extra.csv', index=False)

In [ ]:
sentiment_tb_df.head(5)